In [1]:
DOMAIN = 'test' # test, spouse, cdr

In [2]:
import sys
import os

if DOMAIN == 'cdr':
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 65


In [4]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [5]:
# c = sv.get_selected()
# print c

In [6]:
# hash(c)

In [7]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'cdr':
    user_lists = {}

In [8]:
# examples[0].display()

In [9]:
# examples[28].candidate[0].get_parent()

In [10]:
%time from snorkel.semantic import SemanticParser

CPU times: user 1.61 s, sys: 929 ms, total: 2.54 s
Wall time: 2.95 s


In [11]:
%time sp = SemanticParser(candidate_class, user_lists, absorb=True)

Created grammar with 204 rules
CPU times: user 5.21 ms, sys: 9.35 ms, total: 14.6 ms
Wall time: 82.9 ms


In [19]:
%time sp.evaluate(examples, show_explanation=True, show_all=False, only=[26])

Example 26: label True because the first word to the left of arg 2 equals 'wife'
CPU times: user 19.3 ms, sys: 1.43 ms, total: 20.7 ms
Wall time: 55.1 ms


,Correct,Incorrect,Redundant,Failed,Unknown
Example26,0,1,0,0,0


In [13]:
# (.label, .true, (.function, (.startswith, (.string, blue)), (.string, blueberry)))

In [14]:
# sp.grammar.print_grammar()

In [15]:
# sp.grammar.print_chart()

In [16]:
# total_correct = 0
# total_incorrect = 0
# total_failed = 0
# total_redundant = 0
# for i, example in enumerate(examples):
#     if i==20: break
# #     print example.explanation
#     correct = 0
#     incorrect = 0
#     failed = 0
#     redundant = 0
#     semantics = set()
#     parses = sp.parse(example.explanation, 
#                       example.candidate, 
#                       user_lists=user_lists, 
#                       verbose=False, 
#                       return_parses=True)
# 
#     total_correct += correct
#     total_incorrect += incorrect
#     total_failed += failed
#     total_redundant += redundant
#     valid = correct + incorrect
#     total = correct + incorrect + redundant + failed
#     print "Example %d: %d/%d (%d, %d) -> %d" % (i, correct, valid, redundant, failed, total)
# total_valid = total_correct + total_incorrect
# total_total = total_correct + total_incorrect + total_redundant + total_failed
# print "TOTAL: {}/{} ({:.2f}); {} redundant, {} failed".format(
#                                  total_correct, total_valid, 
#                                  float(total_correct)/total_valid, 
#                                  total_redundant, total_failed)
# print "GROWTH (total): {:.2f}".format(float(total_valid + total_faile)/len(examples))
# print "GROWTH (filter fail): {:.2f}".format(float(total_valid)/len(examples))
# print "GROWTH (filter incorrect): {:.2f}".format(float(total_correct)/len(examples))